In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import warnings
warnings.filterwarnings("ignore")
from math import radians, cos, sin, asin, sqrt
from sklearn import linear_model
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [4]:
#Reading data file 
data = pd.read_csv('df_mod.csv')
data.head()

,trip_id,bike_id,start_station,end_station,trip_route_category,start_time,end_time,start_lat,start_lon,end_lat,end_lon,plan_duration,passholder_type
0,59256078,6417,4159,3042,One Way,21/11/2017 11:42:00 AM,21/11/2017 12:37:00 PM,34.145901,-118.11653,34.049301,-118.23880,0,Walk-up
1,60223612,6453,4159,4138,One Way,27/11/2017 8:13:00 AM,27/11/2017 8:41:00 AM,34.145901,-118.11653,34.145691,-118.14823,0,Walk-up
2,58576402,6650,4159,4138,One Way,17/11/2017 4:39:00 PM,17/11/2017 4:57:00 PM,34.145901,-118.11653,34.145691,-118.14823,30,Monthly Pass
3,51293790,12327,4159,4138,One Way,11/10/2017 16:18,11/10/2017 16:33,34.145901,-118.11653,34.145691,-118.14823,30,Monthly Pass
4,50594191,6000,4159,4138,One Way,8/10/2017 11:51,8/10/2017 12:12,34.145901,-118.11653,34.145691,-118.14823,0,Walk-up


In [0]:
#Creating a new column Date from the column "start_time" of the dataframe 
data["Date"] = pd.to_datetime(data["start_time"])

In [0]:
#Creating a copy of data since loading data takes a lot of time
data1 = data.copy()

In [0]:
#Dropping null values
data1 = data1.dropna() 
# Converting float values to interger
data1["end_station"] = data1["end_station"].astype(int)  
data1 = data1.set_index("Date") #setting index to Date column
data1 = data1.sort_index() #Arranging dates 

In [9]:
data1.head()

,trip_id,bike_id,start_station,end_station,trip_route_category,start_time,end_time,start_lat,start_lon,end_lat,end_lon,plan_duration,passholder_type
Date,,,,,,,,,,,,,
2016-01-08 00:12:00,4767292,6127,3034,3055,One Way,1/8/2016 0:12,1/8/2016 0:49,34.042061,-118.26338,34.044159,-118.25158,0,Walk-up
2016-01-08 00:13:00,4767291,6026,3034,3055,One Way,1/8/2016 0:13,1/8/2016 0:49,34.042061,-118.26338,34.044159,-118.25158,0,Walk-up
2016-01-08 00:18:00,4767290,5990,3021,3022,One Way,1/8/2016 0:18,1/8/2016 0:43,34.045609,-118.23703,34.046070,-118.23309,30,Monthly Pass
2016-01-08 00:23:00,4767289,6585,3063,3064,One Way,1/8/2016 0:23,1/8/2016 1:46,34.049198,-118.25283,34.046810,-118.25698,0,Walk-up
2016-01-08 00:25:00,4767288,6069,3063,3064,One Way,1/8/2016 0:25,1/8/2016 1:46,34.049198,-118.25283,34.046810,-118.25698,0,Walk-up


In [0]:
#length of end station is 142 and length of start station is 140
#start_stn = list(set(data1.start_station))
#end_stn = list(set(data1.end_station))
"""As length is not same of these two list, a search method is used to identify which element 
is missing from the start station, hence the next chunk of code is to find
the station number not in the start station list"""
#remove_stn = []
#for i in range(len(end_stn)):
#    if end_stn[i] not in start_stn:
#        remove_stn.append(end_stn[i])
#Updating end station list
#end_stn = [stn for stn in end_stn if stn not in remove_stn]

In [0]:
#index_df =[]
#for i in range(len(data1.end_station)):
#  if data1.end_station[i] == remove_stn[0] or data1.end_station[i] == remove_stn[1]:
#    index_df.append(data1.index[i])
#  data = data1.drop(index_df)

In [0]:
#Removing rows with station number 4110,4143 and 4118 as it is discarded from the analysis
data1 = data1[data1['end_station']!=4110]
data1 = data1[data1['end_station']!=4118]
data1 = data1[data1['start_station']!=4143]

In [0]:
#Setting plan_duration = 0 for passholder_type = Walk-up
data1['plan_duration'].loc[data1['passholder_type'] == "Walk-up"] = 0

#Setting annual pass = flex as both of them are the same
data1['passholder_type'].loc[data1['passholder_type'] == "Annual Pass"] = "Flex Pass"

In [0]:
#Calculating trip duration in minutes
data1['start_time']= pd.to_datetime(data1['start_time']) 
data1['end_time']= pd.to_datetime(data1['end_time']) 
data1['trip_duration_mins'] = (data1.end_time - data1.start_time)/ timedelta(minutes=1)

In [0]:
#Creating a copy of data since it takes time to load data
data2 = data1.copy()

In [0]:
#Create dummy variables for passholder type
one_hot_pass = pd.get_dummies(data2['passholder_type']).rename(columns={
            'Flex Pass': 'annual',
            'Monthly Pass': 'monthly',
            'One Day Pass': 'one_day',
            'Walk-up':'walk_up'})

#Create dummy variables for trip route category
one_hot_trip_type = pd.get_dummies(data2['trip_route_category']).rename(columns={
            'Round Trip': 'round_trip',
            'One Way': 'one_way'})

In [0]:
#Calculating Distance traveled 
def calc_dist(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [0]:
vfunc = np.vectorize(calc_dist)
data2['Distance']= vfunc(data2.start_lon,data2.start_lat,data2.end_lon,data2.end_lat)

In [0]:
data2 = pd.concat([data2,one_hot_pass  ,one_hot_trip_type], axis=1)

In [0]:
month = 'month'
week = 'week'
day = 'day'
hour = 'hour'
def regression_data(data,col):
    '''
    Stacking dataset on the basis of start_station
    Args: data = dataframe
          col = column name/ time frame
    Returns: Stacked dataframe with index resetted
    '''
    df = data.copy()
    df["Year"]=df.start_time.dt.year
    if col.lower() == 'month':
        col = col.capitalize()
        df[col] = df.start_time.dt.month
    elif col.lower() == 'week':
        col = col.capitalize()
        df[col] = df.start_time.dt.week
    elif col.lower() == 'day':
        col = col.capitalize()
        df[col] = df.start_time.dt.day
    elif col.lower() == 'hour':
        col = col.capitalize()
        df[col] = df.start_time.dt.hour
    else:
        return print('Unable to extract time from index!!')
    df_sum = df.groupby(by=["start_station","Year",col]).sum()[['Distance', 'trip_duration_mins']]
    df_sum1 = df.groupby(by=["start_station","Year",col]).mean()[['annual', 'monthly','one_day', 'walk_up', 'one_way', 'round_trip']]
    df_count = df.groupby(by=["start_station","Year",col]).count()[['trip_id']]
    df = pd.concat([df_sum,df_sum1 ,df_count], axis=1)
    df = df.reset_index()
    return df

In [0]:
df=data2.copy()
df1=df.loc[(df['annual'] == 1) & (df['one_way'] == 1)]
df1=regression_data(df1, month)
df2=df.loc[(df['monthly'] == 1) & (df['one_way'] == 1)]
df2=regression_data(df2, month)
df3=df.loc[(df['one_day'] == 1) & (df['one_way'] == 1)]
df3=regression_data(df3, month)
df4=df.loc[(df['walk_up'] == 1) & (df['one_way'] == 1)]
df4=regression_data(df4, month)
df5=df.loc[(df['annual'] == 1) & (df['round_trip'] == 1)]
df5=regression_data(df5, month)
df6=df.loc[(df['monthly'] == 1) & (df['round_trip'] == 1)]
df6=regression_data(df6, month)
df7=df.loc[(df['one_day'] == 1) & (df['round_trip'] == 1)]
df7=regression_data(df7, month)
df8=df.loc[(df['walk_up'] == 1) & (df['round_trip'] == 1)]
df8=regression_data(df8, month)

In [0]:
#data3 = grouping(data2,month)
df1.loc[df1['Year']==2017, 'Month'] = df1['Month']+12
df1.loc[df1['Year']==2018, 'Month'] = df1['Month']+24
df2.loc[df2['Year']==2017, 'Month'] = df2['Month']+12
df2.loc[df2['Year']==2018, 'Month'] = df2['Month']+24
df3.loc[df3['Year']==2017, 'Month'] = df3['Month']+12
df3.loc[df3['Year']==2018, 'Month'] = df3['Month']+24
df4.loc[df4['Year']==2017, 'Month'] = df4['Month']+12
df4.loc[df4['Year']==2018, 'Month'] = df4['Month']+24
df5.loc[df5['Year']==2017, 'Month'] = df5['Month']+12
df5.loc[df5['Year']==2018, 'Month'] = df5['Month']+24
df6.loc[df6['Year']==2017, 'Month'] = df6['Month']+12
df6.loc[df6['Year']==2018, 'Month'] = df6['Month']+24
df7.loc[df7['Year']==2017, 'Month'] = df7['Month']+12
df7.loc[df7['Year']==2018, 'Month'] = df7['Month']+24
df8.loc[df8['Year']==2017, 'Month'] = df8['Month']+12
df8.loc[df8['Year']==2018, 'Month'] = df8['Month']+24


In [0]:
frames = [df1, df2,df3, df4 , df5, df6, df7,df8]
data3 = pd.concat(frames)

In [0]:
data3

In [0]:
#Setting offset for continuity in months
#data3.loc[data3['Year']==2017, 'Month'] = data3['Month']+12
#data3.loc[data3['Year']==2018, 'Month'] = data3['Month']+24

In [0]:
station = list(set(data3.start_station))
r_squared = []
#Looping over all the stations to fit linear regressions individually
for i in range(len(station)):
    stn = station[i]
    if len(data3.loc[data3['start_station']==stn])>10:
      df = data3.loc[data3['start_station']==stn]
      x = df[['Distance','trip_duration_mins']]
      y = df["trip_id"]
      X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)      
      reg = linear_model.LinearRegression()
      est2 = reg.fit(x,y)
      ypred = est2.predict(X_test) #predicting y values on test dataset
      score = r2_score(y_test,ypred) #Calculating r-squared value on the test data
      r_squared.append(score)
r_squared

In [70]:
len(r_squared)

134

In [73]:
count=0
for i in range(len(r_squared)):
  if r_squared[i]<0.5:
    #if R2[i]==1:
      count = count + 1
print(count)

32
